In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [2]:
df = pd.read_csv('Movielist_Marvel.csv')

df = df.reindex(columns=['name', 'name_link', 
                         'critics_consensus', 'score_critical', 'score_audience', 'count_critical', 'count_audience', 'abstract',
                         'Rating', 'Genre', 'Directed_By', 'Written_By', 'In_Theaters', 'On_Disc_Streaming', 'Box_Office', 'Runtime', 'Studio'])\
                         .astype(str)
df.head()

,name,name_link,critics_consensus,score_critical,score_audience,count_critical,count_audience,abstract,Rating,Genre,Directed_By,Written_By,In_Theaters,On_Disc_Streaming,Box_Office,Runtime,Studio
0,Captain America: The First Avenger,captain_america_the_first_avenger,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Iron Man,iron_man,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,The Incredible Hulk,the_incredible_hulk,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Iron Man 2,iron_man_2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Thor,thor,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [10]:
# check the set of the labels
label_set = set()
for i in range(len(df)):
    time.sleep(1)
    # fetch html by url
    url = 'https://www.rottentomatoes.com/m/' + df['name_link'][i]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    tag = soup.find_all('div',class_="meta-label subtle")
    set_temp = set(map(lambda x: x.text.strip(), tag))
    label_set = label_set.union(set_temp)
    print(i, len(set_temp), df['name_link'][i])
print(len(label_set))
label_set

(0, 9, 'captain_america_the_first_avenger')
(1, 8, 'iron_man')
(2, 9, 'the_incredible_hulk')
(3, 8, 'iron_man_2')
(4, 9, 'thor')
(5, 9, 'marvels_the_avengers')
(6, 9, 'iron_man_3')
(7, 9, 'thor_the_dark_world')
(8, 9, 'captain_america_the_winter_soldier_2014')
(9, 9, 'guardians_of_the_galaxy')
(10, 9, 'guardians_of_the_galaxy_vol_2')
(11, 9, 'avengers_age_of_ultron')
(12, 9, 'antman')
(13, 9, 'captain_america_civil_war')
(14, 9, 'spider_man_homecoming')
(15, 9, 'doctor_strange_2016')
(16, 9, 'thor_ragnarok_2017')
(17, 9, 'black_panther_2018')
(18, 9, 'avengers_infinity_war')
(19, 8, 'ant_man_and_the_wasp')
(20, 7, 'captain_marvel')
9


{u'Box Office:',
 u'Directed By:',
 u'Genre:',
 u'In Theaters:',
 u'On Disc/Streaming:',
 u'Rating:',
 u'Runtime:',
 u'Studio:',
 u'Written By:'}

In [3]:
%%time
for i in range(len(df)):
    print(i)
    time.sleep(1)
    # fetch html by url
    url = 'https://www.rottentomatoes.com/m/' + df['name_link'][i]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    # evaluation
    tag = soup.find_all('p',class_="mop-ratings-wrap__text mop-ratings-wrap__text--concensus")
    df.at[i,'critics_consensus'] = tag[0].text

    tag = soup.find_all('span',class_="mop-ratings-wrap__percentage")
    df.at[i,'score_critical'] = tag[0].text.split('\n')[1].strip()

    tag = soup.find_all('span',class_="mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience")
    df.at[i,'score_audience'] = tag[0].text.split('\n')[1].strip()

    tag = soup.find_all('small',class_="mop-ratings-wrap__text--small")
    df.at[i,'count_critical'] = tag[0].text.split('\n')[2].strip()
    df.at[i,'count_audience'] = tag[1].text.split('\n')[2].strip()

    tag = soup.find_all('div',class_="movie_synopsis clamp clamp-6")
    df.at[i,'abstract'] = tag[0].text.strip()
    
    # meta value
    # labels
    tag = soup.find_all('div',class_="meta-label subtle")
    labels = map(lambda x: x.text.strip(), tag)
    
    # values
    tag = soup.find_all('div',class_="meta-value")
    try:
        index = labels.index('Rating:')
        df.at[i,'Rating'] = tag[index].text.strip()
    except:
        pass

    try:
        index = labels.index('Genre:')
        list_Genre = tag[index].text.replace(',', '').split('\n')
        list_Genre = filter(lambda x : len(x)>0, map(lambda x: x.strip(), list_Genre))
        df.at[i,'Genre'] = list_Genre
    except:
        pass
        
    try:
        index = labels.index('Directed By:')
        list_Directed_By = tag[index].text.replace(',', '').split('\n')
        list_Directed_By = filter(lambda x : len(x)>0, map(lambda x: x.strip(), list_Directed_By))
        df.at[i,'Directed_By'] = list_Directed_By
    except:
        pass
        
    try:
        index = labels.index('Written By:')
        list_Written_By = tag[index].text.replace(',', '').split('\n')
        list_Written_By = filter(lambda x : len(x)>0, map(lambda x: x.strip(), list_Written_By))
        df.at[i,'Written_By'] = list_Written_By
    except:
        pass

    try:
        index = labels.index('In Theaters:')
        df.at[i,'In_Theaters'] = tag[index].text.strip().split('\n')[0]
    except:
        pass

    try:
        index = labels.index('On Disc/Streaming:')
        df.at[i,'On_Disc_Streaming'] = tag[index].text.strip()
    except:
        pass

    try:
        index = labels.index('Box Office:')
        df.at[i,'Box_Office'] = tag[index].text.strip()
    except:
        pass

    try:
        index = labels.index('Runtime:')
        df.at[i,'Runtime'] = tag[index].text.strip()
    except:
        pass

    try:
        index = labels.index('Studio:')
        list_Studio = tag[index].text.replace(',', '').split('\n')
        list_Studio = filter(lambda x : len(x)>0, map(lambda x: x.strip(), list_Studio))
        df.at[i,'Studio'] = list_Studio
    except:
        pass


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
CPU times: user 9.66 s, sys: 20 ms, total: 9.68 s
Wall time: 39.7 s


In [9]:
df.to_csv('BasicInfo_Marvel.csv', encoding='utf-8')
df#.head()

,name,name_link,critics_consensus,score_critical,score_audience,count_critical,count_audience,abstract,Rating,Genre,Directed_By,Written_By,In_Theaters,On_Disc_Streaming,Box_Office,Runtime,Studio
0,Captain America: The First Avenger,captain_america_the_first_avenger,"With plenty of pulpy action, a pleasantly retr...",80%,74%,262,"187,978",Captain America: The First Avenger will focus ...,PG-13 (for intense sequences of sci-fi violenc...,"[Action & Adventure, Mystery & Suspense]",[Joe Johnston],"[Christopher Markus, Stephen McFeely]","Jul 22, 2011","Oct 25, 2011","$176,636,816",124 minutes,[Paramount Pictures]
1,Iron Man,iron_man,"Powered by Robert Downey Jr.'s vibrant charm, ...",93%,91%,274,"1,080,703",Billionaire industrialist and genius inventor ...,PG-13 (for some intense sequences of sci-fi ac...,"[Action & Adventure, Science Fiction & Fantasy]",nan,"[Matthew Hollaway, Mark Fergus, Hawk Ostby, Ar...","Apr 14, 2008","Sep 30, 2008","$318,298,180",126 minutes,[Paramount Pictures]
2,The Incredible Hulk,the_incredible_hulk,The Incredible Hulk may not be quite the smash...,67%,70%,227,"738,333",Mild-mannered scientist Bruce Banner has been ...,PG-13 (for sequences of intense action violenc...,"[Action & Adventure, Science Fiction & Fantasy]",[Louis Leterrier],[Zak Penn],"Jun 12, 2008","Oct 21, 2008","$134,518,390",112 minutes,[Universal Pictures]
3,Iron Man 2,iron_man_2,It isn't quite the breath of fresh air that Ir...,73%,71%,287,"479,041","In ""Iron Man 2,"" the world is aware that billi...",PG-13 (for sequences of intense sci-fi action ...,"[Action & Adventure, Science Fiction & Fantasy]",nan,[Justin Theroux],"May 7, 2010","Sep 28, 2010","$312,057,433",124 minutes,[Paramount Studios]
4,Thor,thor,A dazzling blockbuster that tempers its sweepi...,77%,76%,281,"246,245",The epic adventure Thor spans the Marvel Unive...,PG-13 (for sequences of intense sci-fi action ...,"[Action & Adventure, Drama, Science Fiction & ...",[Kenneth Branagh],"[Zack Stentz, Ashley Edward Miller, Mark Proto...","May 6, 2011","Sep 13, 2011","$181,015,141",115 minutes,[Paramount Pictures]
5,The Avengers,marvels_the_avengers,Thanks to a script that emphasizes its heroes'...,92%,91%,343,"1,133,574",Marvel Studios presents Marvel's The Avengers-...,PG-13 (for intense sequences of sci-fi violenc...,"[Action & Adventure, Science Fiction & Fantasy]",[Joss Whedon],[Joss Whedon],"May 4, 2012","Sep 25, 2012","$623,279,547",142 minutes,[Walt Disney Pictures]
6,Iron Man Three,iron_man_3,"With the help of its charismatic lead, some im...",79%,78%,311,"483,866","Marvel's ""Iron Man 3"" pits brash-but-brilliant...",PG-13 (for sequences of intense sci-fi action ...,"[Action & Adventure, Science Fiction & Fantasy]",[Shane Black],"[Drew Pearce, Shane Black]","May 3, 2013","Sep 24, 2013","$408,992,272",109 minutes,[Walt Disney Pictures]
7,Thor: The Dark World,thor_the_dark_world,It may not be the finest film to come from the...,66%,76%,265,"309,522","Marvel's ""Thor: The Dark World"" continues the ...",PG-13 (for sequences of intense sci-fi action ...,"[Action & Adventure, Science Fiction & Fantasy]",[Alan Taylor],"[Christopher Yost, Christopher Markus, Stephen...","Nov 8, 2013","Feb 25, 2014","$206,360,018",112 minutes,[Walt Disney Pictures]
8,Captain America: The Winter Soldier,captain_america_the_winter_soldier_2014,"Suspenseful and politically astute, Captain Am...",90%,92%,287,"280,702","After being deemed unfit for military service,...","PG-13 (for intense sequences of violence, gunp...","[Action & Adventure, Science Fiction & Fantasy]","[Joe Russo, Anthony Russo]","[Stephen McFeely, Christopher Markus]","Apr 4, 2014","Sep 9, 2014","$228,636,083",136 minutes,[Walt Disney Pictures]
9,Guardians of the Galaxy,guardians_of_the_galaxy,Guardians of the Galaxy is just as irreverent ...,91%,92%,312,"253,935","From Marvel, the studio that brought you the g...",PG-13 (for intense sequences of sci-fi violenc...,"[Action & Adventure, Science Fiction & Fantasy]",[James Gunn (II)]